# Enact Tutorial

## Resources

Enact is a framework for writing generative software. At the core of the
framework is the notion of a `Resource`, which represents a JSON serializable
data object with named fields and field values. Examples of resources include:
* Input and output data,
* callable components called `Invokables`, e.g., a representation of an API
  call into an LLM, together with parameters such as temperature settings,
* call traces of executions called `Invocations`.

Custom resources can be defined by extending the `enact.Resource` class,
which is implemented as a python dataclass:

In [1]:
import enact

import dataclasses

@dataclasses.dataclass
class MyResource(enact.Resource):
  my_field: str
  my_other_field: int

print(f'Unique type identifier: {repr(MyResource.type_id())}')
print(f'Fields of the resource: {list(MyResource.field_names())}')

Unique type identifier: '{"digest": "a46cd7bf341ef134fe8fd7e97e564f0f84425c747c3c55850cd599f1249b97f5", "name": "__main__.MyResource"}'
Fields of the resource: ['my_field', 'my_other_field']


As seen above, `Resource` classes are associated with a list of field
names and a type identifier, which consists of a digest of field names 
and a class name. Resource instances expose a simple interface for 
accessing field values:

In [2]:
my_resource = MyResource('hello', 42)
print(my_resource.my_field)
print(my_resource.my_other_field)
print(f'Field values of the resource: {list(my_resource.field_values())}')
print(f'Field items: {list(my_resource.field_items())}')

hello
42
Field values of the resource: ['hello', 42]
Field items: [('my_field', 'hello'), ('my_other_field', 42)]


The `enact.Resource` class is an implementation of the resource base class
`enact.ResourceBase`. In some cases, it can be preferable to directly inherit
from `ResourceBase` and implement the required functions:

In [3]:
class MyIntResource(enact.ResourceBase, int):

  @classmethod
  def from_fields(cls, field_dict) -> 'MyIntResource':
    return cls(field_dict['value'])

  @classmethod
  def field_names(cls):
    yield 'value'

  def field_values(self):
    yield int(self)

MyIntResource(5).to_resource_dict()

{'value': 5}

Resources may be nested and they can contain maps and lists, but the
leaf types are limited to:
* the Python primitives of type `int`, `float`, `bool`, `str`, `bytes`, `bool`,
* the `None` value,
* other resources (i.e., instances of `ResourceBase`),
* types of resources (i.e., subclasses of `ResourceBase`).

Python maps and lists are allowed and may be nested, but map keys must be of type `str`.

In [4]:
from typing import Dict, List, Union, Type

@dataclasses.dataclass
class ComplexResource(enact.Resource):
  primitives: List[Union[int, float, bool, str, bytes, bool, type(None)]]
  map_value: Dict[str, MyResource]
  nested_resource: MyResource
  resource_type: Type[MyResource]

ComplexResource(
  primitives=[1, 2.0, False, None, 'test', b'bytes'],
  map_value={'hello': MyResource('hello', 69)},
  nested_resource=MyResource('hola', 42),
  resource_type=MyResource).to_resource_dict()

{'primitives': [1, 2.0, False, None, 'test', b'bytes'],
 'map_value': {'hello': {'my_field': 'hello', 'my_other_field': 69}},
 'nested_resource': {'my_field': 'hola', 'my_other_field': 42},
 'resource_type': __main__.MyResource}

Violating these type constraints will lead to runtime errors:

In [5]:
class NotAResource:
  pass

@dataclasses.dataclass
class InvalidResource(enact.Resource):
  x: NotAResource

try:
  InvalidResource(NotAResource()).to_resource_dict()
except enact.FieldTypeError as e:
  print(f'Got error: {e}')

Got error: Encountered unsupported field type <class '__main__.NotAResource'>: <__main__.NotAResource object at 0x7f902cbc14c0>


## Registering and serializing resources

Resources can be serialized to, and deserialized from JSON. When
serializing, a reference to the type id of the resource is stored
in the output. In order to be able to deserialize resources, we need
to register resource types with the framework, which can be 
accomplished with the `enact.register` decorator.

In [6]:
@enact.register
@dataclasses.dataclass
class ResourceToSerialize(enact.Resource):
  value: int

# After registration we can look up the type by its type_id:
print(enact.Registry.get().lookup(ResourceToSerialize.type_id()))

serialized = enact.JsonSerializer().serialize(
    ResourceToSerialize(42).to_resource_dict())
print(f'serialized: {serialized}')

# Since the resource is registered, we can deserialize it without specifying its
# type.
print(enact.JsonSerializer().deserialize(serialized).to_resource())

<class '__main__.ResourceToSerialize'>
serialized: b'{"#res": "{\\"digest\\": \\"828bfb94333b141b83518e5a89ee13babbee6ebee3a9b4f52e692400ce31d722\\", \\"name\\": \\"__main__.ResourceToSerialize\\"}", "value": 42}'
ResourceToSerialize(value=42)


## Resource stores and references

The enact framework allows committing resources to a store, which will return a special
type of resource called a reference. References refer to a resource via a
cryptographic hash digest, that comprises its type information as well as its
contents. This means that two identical instances of a resource will return the
same reference.

Using a store is accomplished by entering its context, which will make it the active
store instance. The `enact.commit` function will commit a resource to the active store
and return a reference.

In [7]:
# Create an ephemeral in-memory store:
with enact.Store() as store:
  my_resource = MyResource('hello', 42)
  identical_resource = MyResource('hello', 42)
  different_resource = MyResource('hello', 43)
  
  ref1: enact.Ref[MyResource] = enact.commit(my_resource)
  ref2: enact.Ref[MyResource] = enact.commit(identical_resource)
  print(ref1)
  print(ref2)
  print(f'ref1 and ref2 are identical? {ref1 == ref2}')
  ref3: enact.Ref[MyResource] = enact.commit(different_resource)
  print(ref3)
  print(f'ref1 and ref3 are identical? {ref1 == ref3}')

<Ref: 947051f773d0e01ed76be7506fe4cfde5be54d35eef91f10b488d3e8b789ee3b>
<Ref: 947051f773d0e01ed76be7506fe4cfde5be54d35eef91f10b488d3e8b789ee3b>
ref1 and ref2 are identical? True
<Ref: e3c81261835f94086c039fadeed74169a6e3afc70733f003ea4484dc3b7ffa79>
ref1 and ref3 are identical? False


A `Ref` object `r` can be dereferenced using the `r.get()` function
or, more simply, by simply calling `r()`.

In [8]:
with store:
  # Must enter context where resource is stored.
  print(ref1.get())
  # This is equivalent to the above:
  print(ref1())

MyResource(my_field='hello', my_other_field=42)
MyResource(my_field='hello', my_other_field=42)


Please be aware that that any modifications to a dereferenced resource will not
alter the original resource to which the reference points. Conceptually,
dereferencing returns a copy of the referenced resource.

In order to alter which resource a reference points at, the `modify` context
manager may be used:

In [9]:
with store:
  print(f'Referenced resource:\n  {ref1()}\n')
  resource: MyResource = ref1()
  resource.my_field = 'changed'
  print(f'Referenced resource after local change:\n  {ref1()}\n')
  
  with ref1.modify() as resource:
    resource.my_field = 'changed again'

  print(f'Reference to new resource after call to modify:\n  {ref1()}')

Referenced resource:
  MyResource(my_field='hello', my_other_field=42)

Referenced resource after local change:
  MyResource(my_field='hello', my_other_field=42)

Reference to new resource after call to modify:
  MyResource(my_field='changed again', my_other_field=42)


### Store backends

A store is connected to a backend. The default backend is
`enact.InMemoryBackend`, which means that the store has the same lifetime as the
process.


For persistent storage in the file system, the `FileBackend`
can be used:

In [10]:
import tempfile

# File storage requires that we register the types we want to store
# since serialization and deserialization is required to store the data.
enact.register(MyResource)

with tempfile.TemporaryDirectory() as root_dir:
  with enact.Store(enact.FileBackend(root_dir)) as file_store:
    ref = enact.commit(MyResource('hello', 42))

  del file_store  # Remove reference to file store

  with enact.Store(enact.FileBackend(root_dir)) as file_store:
    # The resource is still available.
    print(ref())

MyResource(my_field='hello', my_other_field=42)


New storage backends can be created by implementing the following interface:

```python
class StorageBackend(abc.ABC):
  """A storage backend."""

  @abc.abstractmethod
  def commit(self, packed_resource: PackedResource):
    """Stores a packed resource."""

  @abc.abstractmethod
  def has(self, ref: Ref) -> bool:
    """Returns whether the storage backend has the resource."""

  @abc.abstractmethod
  def get(self, ref: Ref) -> Optional[interfaces.ResourceDict]:
    """Returns the packed resource or None if not available."""
```

Resources are _packed_ into serializable dictionaries before being committed.
This additional step can be overriden in custom `Ref` subclasses in order to
support advanced functionality such as compression or encryption of resources.

### Stores are hash DAGs

Stores contain resources that may contain references to other resources. They
can therefore be interpreted as directed graphs.

Note that it is not possible to commit two resources that reference each other:
Taking a reference requires creating a cryptographic digest of the contents of
the resource, including all of its own references. Therefore, to compute the
digest of the first resource, we would need to first compute the digest of
the second - but this would in turn require computing the digest of the first.

Since resources can't reference each other we can conclude that the graph is
acyclic, which means that they form a _directed acyclic graph_ (DAG). DAGs that
employ this form of hashing are known as Merkle DAGs or hash DAGs. Hash DAGs are
generalizations of data structures such as Merkle trees and block chains.

## Invokables and Invocations

`Invokables` are special types of resources that define a `call` function and
may optionally be annotated with input and output resource types using the 
`enact.typed_invokable(input_type, output_type)` decorator. Invokables are
callable objects and users should take care to execute them via
`my_invokable(input)` rather than via `my_invokable.call(input)`

In [11]:
@enact.typed_invokable(enact.Str, enact.Str)  # Will also register the resource.
@dataclasses.dataclass
class HelloEnact(enact.Invokable):
  greeting: str = 'Hello'

  def call(self, name: enact.Str) -> enact.Str:
    # enact.Str wraps python str in a Resource interface.
    return enact.Str(f'{self.greeting} {name}!')

greeter = HelloEnact()
greeter('World')

'Hello World!'

Similar to the case of resources, `enact.Invokable` is a convenience
implementation of the `enact.InvokableBase` base class using dataclasses, and it
may be preferable to directly inherit from the latter in some situations.

Invokables allow for _tracked executions_ that return a special `Invocation`
resource. Invocation must be done in the context of a store and expects
a reference to an input:

In [12]:
with store:
  input_ref = enact.commit(enact.Str('World'))
  invocation = greeter.invoke(input_ref)
  enact.pprint(invocation)

Invocation:
  request:
    -> Request#33f761:
      invokable: -> HelloEnact(greeting='Hello')#b408cb
      input: -> 'World'#ae44ee
  response:
    -> Response#1f41be:
      invokable: -> HelloEnact(greeting='Hello')#b408cb
      output: -> 'Hello World!'#72ff74
      raised: None
      raised_here: False
      children: []


Note that invoking a resource will recursively track subinvocations; that is, an
invocation is a full record of the input, output and any raised exceptions of
invokables that participated in the call.

In [13]:
import random

@enact.typed_invokable(enact.NoneResource, enact.Str)
@dataclasses.dataclass
class Sampler(enact.Invokable):
  options: List[str]

  def call(self):
   return enact.Str(random.choice(self.options))

@enact.typed_invokable(enact.Str, enact.Str)
@dataclasses.dataclass
class SampleAndGreet(enact.Invokable):
  greeting_sampler: Sampler

  def call(self, name: enact.Str):
    return enact.Str(self.greeting_sampler() + ' ' + name)

with store:
  sampler = Sampler(options=['Hello', 'Hi', 'Greetings'])
  sample_and_greet = SampleAndGreet(greeting_sampler=sampler)
  input_ref = enact.commit(enact.Str('World'))
  invocation = sample_and_greet.invoke(input_ref)
  print(f'Output: {invocation.response().output()}')
  enact.pprint(invocation)

Output: Hello World
Invocation:
  request:
    -> Request#a9e685:
      invokable:
        -> SampleAndGreet#7f701a:
          greeting_sampler:
            Sampler:
              options:
                [
                  'Hello'
                  'Hi'
                  'Greetings']
      input: -> 'World'#ae44ee
  response:
    -> Response#26ab61:
      invokable:
        -> SampleAndGreet#7f701a:
          greeting_sampler:
            Sampler:
              options:
                [
                  'Hello'
                  'Hi'
                  'Greetings']
      output: -> 'Hello World'#8d9c46
      raised: None
      raised_here: False
      children:
        [
          -> Invocation#486f38:
            request:
              -> Request#fe5514:
                invokable:
                  -> Sampler#e4a93e:
                    options:
                      [
                        'Hello'
                        'Hi'
                        'Greetings']
                

### Replaying invocations

In generative software, it can be useful to partially replay previous
executions, e.g., to reuse certain generative outputs but resample others.  To
this end, invocations support a replay functionality. By default, a replay will
reuse previous execution outputs rather than reexecuting them (additionally, it
will attempt to replay any modifications to the invokable during the call), but
if a previous invocation raised an exception, or if the invocation is missing an
output, it will reexecute the invokable:

In [14]:
@enact.typed_invokable(enact.NoneResource, enact.Int)
@dataclasses.dataclass
class RollDie(enact.Invokable):
  """Rolls a die and returns the result."""
  sides: int = 6

  def call(self) -> enact.Int:
    return enact.Int(random.randint(1, self.sides))

@enact.typed_invokable(enact.NoneResource, enact.Int)
@dataclasses.dataclass
class RollTwice(enact.Invokable):
  """Rolls a die twice and returns the sum."""
  sides: int = 6

  def call(self) -> enact.Int:
    return enact.Int(RollDie(sides=self.sides)() +
                     RollDie(sides=self.sides)())

with store:
  invocation = RollTwice().invoke()
  print(f'Roll 1: {invocation.response().children[0]().get_output()}')
  print(f'Roll 2: {invocation.response().children[1]().get_output()}')
  print(f'Result: {invocation.get_output()}')

  with invocation.response.modify() as response:
    # Remove the output of the top level invocation to trigger
    # recomputation of RollTwice.
    response.output = None
    with response.children[1].modify() as child:
      # Remove the output of the second dice roll to trigger
      # recomputation of the the second call to RollDie.
      with child.response.modify() as response:
        response.output = None

  print(f'\n=== Partial replay:')
  # Replay the partial invocation.
  invocation = invocation.replay()
  print(f'Replayed roll 1: {invocation.response().children[0]().get_output()}')
  print(f'Rerolled roll 2: {invocation.response().children[1]().get_output()}')
  print(f'Result: {invocation.get_output()}')


Roll 1: 2
Roll 2: 6
Result: 8

=== Partial replay:
Replayed roll 1: 2
Rerolled roll 2: 3
Result: 5


### Exception overrides

The invoke function supports an `exception_override` argument, which allows the
injection of resources during a replay in place of an exception that was
previously raised. This mechanism can be used to halt an execution to query an
external user or data source and then resume with the new value.

In [15]:
class HumanRequired(enact.ExceptionResource):
  pass

@enact.typed_invokable(enact.NoneResource, enact.Str)
class AskUser(enact.Invokable):
  def call(self):
    raise HumanRequired('I am not a user')

with store:
  greeter = SampleAndGreet(AskUser())
  input_ref = enact.commit(enact.Str('World'))
  invocation = greeter.invoke(input_ref)
  print(f'Raised: {repr(invocation.response().raised())}')

  def exception_override(exception_ref):
    if isinstance(exception_ref(), HumanRequired):
      print('Injecting value \'Hi\' into replay')
      return enact.Str('Hi')

  invocation = invocation.replay(
      exception_override=exception_override)
  print(invocation.get_output())
  

Raised: HumanRequired('I am not a user')
Injecting value 'Hi' into replay
Hi World


There is special support in enact to simplify this type of flow.
The `request_input` function can be used to raise an `InputRequired`
exception, which provides a convenient `continue_invocation` function
to resume the replay:

In [16]:
@enact.typed_invokable(enact.NoneResource, enact.Str)
class ConcatUserInputs(enact.Invokable):
  def call(self):
    str1 = enact.request_input(enact.Str, context='Please provide input')
    str2 = enact.request_input(
      enact.Str,
      for_resource=str1,
      context='Please provide a string to append to the previous input')
    return enact.Str(str1 + str2)

with store:
  concat = ConcatUserInputs()
  invocation = concat.invoke()
  print('== First input request:')
  print(f'Context: {invocation.get_raised().context}')
  invocation = invocation.get_raised().continue_invocation(
    invocation,
    enact.Str('Hello '))
  print('== Second input request:')
  print(f'Context: {invocation.get_raised().context}')
  print(f'Input: {invocation.get_raised().input()}')
  invocation = invocation.get_raised().continue_invocation(
    invocation,
    enact.Str('World'))
  print('== Result:')
  enact.pprint(invocation.get_output())


== First input request:
Context: Please provide input
== Second input request:
Context: Please provide a string to append to the previous input
Input: Hello 
== Result:
'Hello World'


### Replays and determinism

Replays require that the only parts of the code that behave
non-deterministically (in the sense of depending on system
state or randomness) are wrapped in `Invokable` objects. Python
level non-determinism raises an error during replays:

In [17]:
import time

@enact.typed_invokable(enact.Str, enact.Str)
class BadGreeter(enact.Invokable):
  """A bad greeter that executes non-deterministically."""

  def call(self, name: enact.Str) -> enact.Str:
    """Call the greeter."""
    # Non-determinism originating from a python function rather than a
    # subinvocation:
    greeting = f'Current time: {time.time()}. Hello'
    greeter = HelloEnact(greeting)
    return greeter(name)

with store:
  input_ref = enact.commit(enact.Str('World'))
  invocation = BadGreeter().invoke(input_ref)
  print(invocation.get_output())
  
  # Force reexecution of the top-level invokable by deleting its output.
  with invocation.response.modify() as response:
    response.output = None
  
  try:
    invocation = invocation.replay()
  except enact.ReplayError as e:
    print(f'Got replay error: {e}')

Current time: 1690854863.451003. Hello World!
Got replay error: Expected invocation HelloEnact(greeting='Current time: 1690854863.4521444. Hello')(World) but got HelloEnact(greeting='Current time: 1690854863.451003. Hello')(World).
Ensure that all non-deterministic functions are wrapped in invokables or use strict=False.


The code above can be fixed by introducing an explicit invokable to determine
current time. This allows the replay to reuse the previously generated output.

In [18]:
@enact.typed_invokable(enact.NoneResource, enact.Float)
class CurrentTime(enact.Invokable):
  """Returns the current time in seconds since epoch."""

  def call(self) -> enact.Float:
    """Return the time."""
    return enact.Float(time.time())

@enact.typed_invokable(enact.Str, enact.Str)
class TimeGreeter(enact.Invokable):
  """A greeter that includes the current time."""

  def call(self, name: enact.Str) -> enact.Str:
    """Call the greeter."""
    timer = CurrentTime()  # Non-determinism wrapped inside invokable.
    greeting = f'Current time: {timer()}. Hello'
    greeter = HelloEnact(greeting)
    return greeter(name)

with store:
  input_ref = enact.commit(enact.Str('World'))
  invocation = TimeGreeter().invoke(input_ref)
  print(invocation.get_output())
  
  # Force reexecution of the top-level invokable by deleting its output.
  with invocation.response.modify() as response:
    response.output = None
  invocation = invocation.replay()
  print(invocation.get_output())

Current time: 1690854863.501931. Hello World!
Current time: 1690854863.501931. Hello World!
